This just ellucidates and confirms that our effnet is doing what we expected

In [2]:
from model import EffNet

import torch
import numpy as np
import pandas as pd
import os
import h5py
from exabiome.nn.loader import read_dataset, LazySeqDataset, train_test_loaders
import argparse
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from typing import Type, Any, Callable, Union, List, Optional

ImportError: attempted relative import with no known parent package

In [2]:
x = torch.randint(0, 19, (64,4096))
x.shape

torch.Size([64, 4096])

In [4]:
emb_out = nn.Embedding(20,8)(x)
emb_out.shape

torch.Size([64, 4096, 8])

In [5]:
emb_out.permute(0,2,1).shape

torch.Size([64, 8, 4096])

In [6]:
nn.Conv1d(8, 32, 2)(emb_out.permute(0,2,1)).shape

torch.Size([64, 32, 4095])

In [2]:
hparams = argparse.Namespace(**{'load': False,
                            'window': 4096,
                            'step': 4096,
                             'classify': False,
                               'tgt_tax_lvl': "species",
                               'fwd_only': True,
                                #'input':path,
                               'manifold': True})

In [5]:
path = '/global/homes/a/azaidi/scratch/azaidi/taxon_trials/ar122_r202.rep.h5'
#path = 'global/cscratch1/sd/azaidi/taxon_trials/ar122_r202.rep.h5'
chunks = LazySeqDataset(hparams=hparams, 
                        path=path,
                       keep_open=True)


In [5]:
x = batch[1]
y = batch[2]
x.shape, y.shape

(torch.Size([16, 4096]), torch.Size([16, 16]))

In [7]:
model = EffNet_b0(out_feats=1, n_classes=16)

NameError: name 'EffNet_b0' is not defined

In [7]:
out = model(x.unsqueeze(1).to(torch.float))
out.shape, y.shape

(torch.Size([16, 16, 1]), torch.Size([16, 16]))

Looks like the forward pass does what's expected - now to just confirm with the loss function

In [ ]:
class DistMSELoss(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, output, target):
        x2 = output.pow(2).sum(axis=1)
        xy = 2*output.mm(output.T)
        dist = (((x2 - xy).T + x2))
        n = output.shape[0]
        loss = (dist - target).pow(2).sum()/(n*(n-1))
        return loss

In [12]:
DistMSELoss()(out.squeeze(2), y)

NameError: name 'DistMSELoss' is not defined